# Web scraping REPS con Selenium.

Que son los REPS?: Registro Especial de Prestadores de Servicios de Salud -REPS. Es la base de datos de las Entidades Departamentales y Distritales de Salud, en la cual se efectúa el registro de los Prestadores de Servicios de Salud que se encuentren habilitados y es consolidada por parte del Ministerio de Salud y Protección Social [**link**](https://www.minsalud.gov.co/sites/rid/Lists/BibliotecaDigital/RIDE/DE/PES/caracterizacion-registro-especial-prestadores-reps.pdf)

Que funcionalidad hace este script: Se encarga de descargar los archivos de:
- Prestadores
- Sedes
- Servicios
- Capacidad

desde la pagina web.

In [1]:
import time
start_i = time.time()

In [2]:
from glob import glob
from lxml import etree
from selenium import webdriver
import pandas as pd
import datetime, os, gc

In [3]:
def clean_not_downloaded(download_path:str, debug:bool=False):
    if debug==True: print(f"fx({clean_not_downloaded.__name__})")
    downloading_chrome  = glob(download_path+'\\*.crdownload')
    downloading_firefox = glob(download_path+'\\*.part')
    files = downloading_chrome+downloading_firefox
    if len(files)>0:
        for f in files:
            try:
                os.remove(f)
                if debug==True: print(f"\tfile deleted {f}")
            except:
                pass
    return

In [4]:
def wait_til_download(download_path:str, debug:str=False):
    """
        wait til download files
    """
    if debug==True: print(f"fx({wait_til_download.__name__})")
    downloading = 1
    while downloading>0:
        time.sleep(5)
        downloading_chrome  = glob(download_path+'\\*.crdownload')
        downloading_firefox = glob(download_path+'\\*.part')
        # print(downloading_chrome + downloading_firefox)
        downloading = len(downloading_chrome + downloading_firefox)
    time.sleep(30)
    return

In [5]:
def get_last_downloaded_file(download_path:str, prefix_file:str=None, debug:bool=False):
    if debug==True: print(f"fx({get_last_downloaded_file.__name__})")
    if prefix_file==None:
        path = download_path+'\\*'
    else:
        path = download_path+f'\\{prefix_file}*'
    files = [(f, os.path.getmtime(f)) for f in glob(path)]
    files.sort(key=lambda a: a[1], reverse=True)
    last_file = files[0][0]
    k = os.path.basename(last_file).split(".")[0].split("(")[0].strip()
    return k, last_file

In [6]:
def get_last_files_donloaded(download_path:str, debug:bool=False):
    """
        Get all last files downloaded
    """
    if debug==True: print(f"fx({get_last_files_donloaded.__name__})")
    downloaded_files = dict()
    for k in set([os.path.basename(f).split(".")[0].split("(")[0].strip() for f in glob(download_path+'/*')]):
        _, last_file = get_last_downloaded_file(download_path=download_path, prefix_file=k)
        downloaded_files.update({k: last_file})
    return downloaded_files

In [7]:
def make_download_path(download_path:str=None, debug:bool=False):
    if debug==True: print(f"fx({make_download_path.__name__})")
    if download_path==None:
        download_path = os.path.join(os.getcwd(), "downloads")
    else:
        try:
            os.path.exists(download_path)
        except:
            try:
                os.makedirs(download_path)
            except Exception as e:
                e = f"\tcan't make path.\n\t{download_path}\nError: {e}"
                raise Exception(e)
    return download_path

In [8]:
def make_options(download_path:str, kind_of_driver:str="firefox", debug:bool=False):
    if debug==True: print(f"fx({make_options.__name__})")

    if kind_of_driver=="firefox":
        
        options = webdriver.firefox.options.Options()
        options.set_preference("browser.download.manager.showWhenStarting", False)
        options.set_preference("browser.download.dir", download_path)
        options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")

    elif kind_of_driver=="chrome":
        
        options = webdriver.chrome.options.Options()
        preferences = {
                        "download.default_directory": download_path , # pass the variable
                        "download.prompt_for_download": False,
                        "directory_upgrade": True,
                        "safebrowsing.enabled": True
                    }
        options.add_experimental_option("prefs", preferences)
    
    else:

        e = f"kind_of_driver no es valido [{kind_of_driver}]"
        raise Exception(e)
    
    return options

In [9]:
def start_driver(kind_of_driver:str, options, debug:bool=False):
    if debug==True: print(f"fx({start_driver.__name__})")
    if kind_of_driver=="chrome":
        # driver = webdriver.Chrome(options=options, executable_path=".\drivers\chromedriver")
        try:
            s = webdriver.chrome.service.Service(".\drivers\chromedriver.exe")
            driver = webdriver.Chrome(options=options, service=s)
        except:
            driver = webdriver.Chrome(options=options)
    elif kind_of_driver=="firefox":
        # driver = webdriver.Firefox(firefox_profile=options)
        driver = webdriver.Firefox(options=options)
    else:
        e = f"kind_of_driver no es valido [{kind_of_driver}]"
        raise Exception(e)
    time.sleep(2)
    return driver

In [10]:
def sing_in(driver, url_loggin, debug:bool=False):
    if debug==True: print(f"fx({sing_in.__name__})")
    driver.get(url_loggin)
    time.sleep(3)
    driver.find_element(by="id",value="exampleModal").click()   ### cerrar la ventana emergente
    time.sleep(0.5)
    driver.find_element(by="id",value="Button1").click()        ### ingresar
    time.sleep(0.5)
    return 

In [11]:
def get_download_data(driver, url, debug:bool=False):
    if debug==True: print(f"fx({get_download_data.__name__})")
    driver.get(url)
    time.sleep(0.5)
    driver.find_element(by="id",value="_ctl0_ibBuscarFtr").click()                           ### cargar todos
    time.sleep(2)
    driver.find_element(by="id",value="_ctl0_ContentPlaceHolder1_ibExcel").click()           ### descargar excel
    if debug==True: print("\t",url)
    return

In [12]:
def read_downloaded_file(k:str, f:str, fechaconsulta, debug:bool=False):
    if debug==True: print(f"fx({read_downloaded_file.__name__})")
    try:
        df = pd.read_html(f)
        if debug==True: print("\tReaded by pandas")
        if isinstance(df,pd.DataFrame):
            pass
        elif isinstance(df,list):
            if isinstance(df[0],pd.DataFrame) and df[0].columns[0]==0:
                cols = df[0].iloc[0].values
                df = df[0].iloc[1:]
                df.columns = cols
            else:
                e = f"\tbad structure."
                print(e)
                raise Exception(e)
        else:
            e = f"\tbad structure. not [list] or [pd.DataFrame]"
            print(e)
            raise Exception(e)
    except:
        try:
            with open(f, 'rt') as fo:
                raw = etree.HTML(fo.read()).find("body/table")
                rows = iter(raw)
                headers = [col.text for col in next(rows)]
                values=[]
                for row in rows:
                    values.append([col.text for col in row])
                df = pd.DataFrame(values,columns=headers)
            if debug==True: print("\tReaded by etree")
        except Exception as e:
            e = f"\tno se leyo.. {f}\nError: {e}"
            raise Exception(e)
    if debug==True: print("\ttype(df) = ", type(df))
    df["fechahora_consulta"] = fechaconsulta
    if debug==True: print(f"\t{k}.shape = {df.shape}")
    return df

In [13]:
def save_to_parquet(k:str, df:pd.DataFrame, fechaconsulta, debug:bool=False):
    if debug==True: print(f"fx({save_to_parquet.__name__})")
    try:
        os.makedirs("output")
    except:
        pass
    fName = f'output/{k.lower()}__{fechaconsulta.strftime("%Y%m%d_%H%M%S%f")}.parquet'
    df.to_parquet(fName)
    return fName

## main

In [14]:
url                 = "https://prestadores.minsalud.gov.co/habilitacion"
url_loggin          = f"{url}/work.aspx"
url_prestadores     = f"{url}/consultas/habilitados_reps.aspx"
url_sedes           = f"{url}/consultas/sedes_reps.aspx"
url_servicios       = f"{url}/consultas/serviciossedes_reps.aspx"
url_capacidad       = f"{url}/consultas/capacidadesinstaladas_reps.aspx"
urls                = [url_prestadores, url_sedes, url_servicios, url_capacidad]
kind_of_driver      = "chrome"
downloaded_files    = list()

debug = False    # True

In [ ]:
download_path = make_download_path()
clean_not_downloaded(download_path=download_path)
download_path


In [16]:
%%time
options = make_options(download_path=download_path, kind_of_driver=kind_of_driver, debug=debug)
driver = start_driver(kind_of_driver=kind_of_driver, options=options, debug=debug)
print(f"Driver started ... {kind_of_driver}")

Driver started ... chrome
CPU times: total: 15.6 ms
Wall time: 4.08 s


In [17]:
%%time
sing_in(driver=driver, url_loggin=url_loggin, debug=debug)
print(f"Logging in web ... {url_loggin}")

Logging in web ... https://prestadores.minsalud.gov.co/habilitacion/work.aspx
CPU times: total: 0 ns
Wall time: 5.92 s


In [18]:
%%time
for u in urls:
    # u = urls[0]
    print("*"*50, u)
    start = time.time()
    get_download_data(driver=driver, url=u, debug=debug)
    fechaconsulta = datetime.datetime.now()
    wait_til_download(download_path=download_path, debug=debug)
    k,last_file = get_last_downloaded_file(download_path=download_path, debug=debug)
    downloaded_files.append({k: {"file": last_file, "url":u, "fechahora_consulta": fechaconsulta}})
    time.sleep(5)
    print("\t", round((time.time()-start)/60,2),"min.")
    print("")
driver.close()
driver.quit()

************************************************** https://prestadores.minsalud.gov.co/habilitacion/consultas/habilitados_reps.aspx
	 1.08 min.

************************************************** https://prestadores.minsalud.gov.co/habilitacion/consultas/sedes_reps.aspx
	 1.46 min.

************************************************** https://prestadores.minsalud.gov.co/habilitacion/consultas/serviciossedes_reps.aspx
	 5.52 min.

************************************************** https://prestadores.minsalud.gov.co/habilitacion/consultas/capacidadesinstaladas_reps.aspx
	 1.79 min.

CPU times: total: 141 ms
Wall time: 9min 53s


In [19]:
try:
    if len(downloaded_files)==0: raise Exception
except:
    downloaded_files = get_last_files_donloaded(download_path)
downloaded_files

[{'Prestadores': {'file': 'd:\\dherrera\\OneDrive - COOSALUD EPS-S\\PERSONAL\\LearnigDS-ML\\WebScraping\\downloads\\Prestadores.xls',
   'url': 'https://prestadores.minsalud.gov.co/habilitacion/consultas/habilitados_reps.aspx',
   'fechahora_consulta': datetime.datetime(2023, 1, 25, 14, 57, 31, 205343)}},
 {'Sedes': {'file': 'd:\\dherrera\\OneDrive - COOSALUD EPS-S\\PERSONAL\\LearnigDS-ML\\WebScraping\\downloads\\Sedes.xls',
   'url': 'https://prestadores.minsalud.gov.co/habilitacion/consultas/sedes_reps.aspx',
   'fechahora_consulta': datetime.datetime(2023, 1, 25, 14, 58, 43, 548117)}},
 {'Servicios': {'file': 'd:\\dherrera\\OneDrive - COOSALUD EPS-S\\PERSONAL\\LearnigDS-ML\\WebScraping\\downloads\\Servicios.xls',
   'url': 'https://prestadores.minsalud.gov.co/habilitacion/consultas/serviciossedes_reps.aspx',
   'fechahora_consulta': datetime.datetime(2023, 1, 25, 15, 1, 4, 392393)}},
 {'CapacidadInstalada': {'file': 'd:\\dherrera\\OneDrive - COOSALUD EPS-S\\PERSONAL\\LearnigDS-ML\\W

In [20]:
# read and save in parquet
for x in downloaded_files:
    start = time.time()
    k = list(x.keys())[0]
    f = list(x.values())[0]["file"]
    fechaconsulta = list(x.values())[0]["fechahora_consulta"]
    print("*"*50, k)
    df = read_downloaded_file(k=k, f=f, fechaconsulta=fechaconsulta, debug=debug)
    save_to_parquet(k=k, df=df, fechaconsulta=fechaconsulta, debug=debug)
    t = time.time()-start
    if t>60:
        t = f"{round((t/60),1)}min."
    else:
        t = f"{round(t,0)}seg."
    print(f"\teplased time = {t}")

************************************************** Prestadores
	eplased time = 34.0seg.
************************************************** Sedes
	eplased time = 59.0seg.
************************************************** Servicios
	eplased time = 2.1min.
************************************************** CapacidadInstalada
	eplased time = 53.0seg.


In [21]:
print("END... ", round((time.time()-start_i)/60,1),"min.")

END...  14.6 min.
